In [1]:
# load
import pandas as pd
import lightgbm

data = pd.read_csv("X_train.csv", index_col=0)
data["mark"] = pd.read_csv("y_train.csv", index_col=0)["mark"]

stud_info = pd.read_csv("studs_info.csv", index_col=False)

X_validation = pd.read_csv("X_test.csv", index_col=0)

In [2]:
# rename columns
field_map = {
    "STD_ID": "stud",
    "НАПРАВЛЕНИЕ": "profession",
    "ГОД": "year",
    "АТТЕСТАЦИЯ": "exam_type",
    "ДИСЦИПЛИНА": "discipline",
    "КУРС": "course",
    "СЕМЕСТР": "semester",

    "   number": "number",
    "Пол": "sex",
    "Статус": "state",
    "Дата выпуска": "release_date",
    "Категория обучения": "category",
    "Форма обучения": "study_kind",
    "Шифр": "cipher",
    "направление (специальность)": "speciality",
    "   ": "what?",
    "Образование": "lvl_of_education",
    "Дата выдачи": "issue_date",
    "Что именно закончил": "education",
}

data.rename(columns=field_map, inplace=True)
X_validation.rename(columns=field_map, inplace=True)
stud_info.rename(columns=field_map, inplace=True)

In [3]:
stud_info[stud_info["stud"] == 182383]

,stud,number,sex,state,release_date,category,study_kind,cipher,speciality,what?,lvl_of_education,issue_date,education
3582,182383,348.0,М,СТ,NaN,ДОГ,З,11.03.03,Конструирование и технология электронных средств,966.0,ОСН,2010-06-17 00:00:00,"СРЕДНЯЯ ШКОЛА N 18, НОВГОРОД"
3583,182383,348.0,М,СТ,NaN,ДОГ,З,11.03.03,Конструирование и технология электронных средств,9284.0,СП,2014-03-04 00:00:00,Политехнический колледж НовГУ


In [4]:
stud_info.drop(stud_info[stud_info["stud"] == 92222].index, inplace=True)

# stud_info[np.isin(stud_info["number"], range(850, 900))].sort_values(by=["stud"])
# all(stud_info.groupby("speciality")["cipher"].nunique().eq(1))# and
all(stud_info.groupby("cipher")["speciality"].nunique().eq(1))
g = stud_info.groupby("speciality")["cipher"].nunique()
print(g[g != 1])

set(stud_info[stud_info["speciality"] == "Журналистика"]["cipher"])
# 203283

speciality
Журналистика                                                                 2
Зоотехния                                                                    2
Конструирование и технология электронных средств                             2
Конструкторско-технологическое обеспечение машиностроительных производств    2
Менеджмент                                                                   2
Прикладная математика и информатика                                          2
Радиотехника                                                                 2
Строительство                                                                2
Филология                                                                    2
Электроника и наноэлектроника                                                2
Юриспруденция                                                                2
Name: cipher, dtype: int64


{'42.03.02', '42.04.02'}

In [5]:
%%capture
# clean up

# for each stud: year == course + const
# for each stud: course == ceil(semester / 2)
# therefore they are noise
fields = ["year", "course"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

# all nulls and not present in data / validation
stud_info.drop(stud_info[stud_info["stud"] == 92222].index, inplace=True)

# for each stud: all number_s are equal
assert all(stud_info.groupby("number")["stud"].nunique().le(1)) and all(stud_info.groupby("stud")["number"].nunique().le(1))
fields = ["number", "issue_date", "release_date"]
stud_info.drop(fields, axis=1, inplace=True)

In [6]:


# # a = pd.DataFrame({"A": [1, 2, 3, 2, 1], "B": ["01", "02", "03", "04", "05"]})
# # b = pd.DataFrame({"A": [1, 2, 3, 4, 5, 6], "C": ["01", "02", "03", "04", "05", "06"]})
# # pd.merge(a, b, how="left", on="A")
# # len(set(stud_info["stud"]) - set(X_validation["stud"]))

# import numpy as np
# mapping = stud_info.groupby("stud")["issue_date"].apply(lambda x: len(x) > 1).to_dict()
# print(mapping[182383])
# g = ~(stud_info["issue_date"].isnull() & stud_info["stud"].map(mapping))
# print(g[stud_info["stud"] == 182383])
# stud_info = stud_info[g]
# print(stud_info[stud_info["stud"] == 182383])
# stud_info = stud_info.sort_values(by=["stud", "issue_date"])

# ids = stud_info[stud_info.duplicated(subset=["stud"])]["stud"]
# # mapping = stud_info.groupby("stud")["issue_date"].apply(lambda x: np.all(~x.isnull())).to_dict()
# # null_less = stud_info[stud_info["stud"].map(mapping) & np.isin(stud_info["stud"], ids)]
# # last_lvl_of_education = null_less.groupby("stud").last()
# # last_lvl_of_education = stud_info[np.isin(stud_info["stud"], ids)].groupby("stud").last()
# last_lvl_of_education = stud_info.groupby("stud").last()


# g = ~np.isin(stud_info["stud"], last_lvl_of_education.index)
# print(g[stud_info["stud"] == 182383])
# # print(last_lvl_of_education[last_lvl_of_education["stud"] == 182383])
# stud_info = stud_info[g]
# stud_info = pd.concat([stud_info, last_lvl_of_education])

# print(stud_info[stud_info["stud"] == 182383])

# assert len(stud_info[np.isin(stud_info["stud"], stud_info[stud_info.duplicated(subset=["stud"])])]) == 0
# fields = ["issue_date", "release_date"]
# stud_info.drop(fields, axis=1, inplace=True)

# y = 760
# last_lvl_of_education[y:y+30]

# # from collections import Counter

# # Counter(null_less.groupby("stud")["lvl_of_education"].apply(tuple))

# # {
# # # ('НС', 'СР'): 4,
# # #  ('ОСН', 'СР'): 3,
# # #  ('НС', 'СП'): 5,
# # #  ('СР', 'СП'): 111,
# # #  ('ОСН', 'СП'): 24,
# # #  ('ОО', 'СР'): 22,
# # #  ('ОО', 'СП'): 131,
# #  ('НП', 'СР'): 1,
# #  ('НП', 'СП'): 10,
# #  ('СП', 'СП'): 7,
# #  ('СР', 'СР', 'СП'): 1,
# #  ('СР', 'СР'): 1,
# #  ('СП', 'СР'): 1,
# #  ('СП', 'НП'): 1}

# # # ('ОО', 'СР'      )
# # # (      'СР', 'СП')
# # # ('ОО',       'СП')

# # # ('ОО',        'СР', 'СП')
# # # (      'ОСН', 'СР'      )
# # # (      'ОСН', 'СП'      )

# # ('ОО', 'ОСН', 'СР', 'СП')
# # ('НС',        'СР'      )
# # ('НС',              'СП')

# # SeriesGroupBy.cummax()

# # len(stud_info), len(stud_info.drop_duplicates(subset=["stud"]))

# # special = stud_info[np.isin(stud_info["stud"], X_validation["stud"])]
# # # # len(set(X_validation["stud"])), len(set(stud_info["stud"])), len(set(special["stud"]))
# # g = pd.merge(X_validation, special, how="left", on="stud", indicator=True)
# # # g[pd.isnull(g["state"])]
# # # pd.concat([X_validation,g[X_validation.columns]]).drop_duplicates(keep=False)
# # # g[X_validation.columns].drop_duplicates(keep=False)
# # g.drop_duplicates(keep=False)

# # X_validation.drop_duplicates(keep=False)
# # df1 = X_validation
# # df2 = g[X_validation.columns]
# # # df1[~df1.apply(tuple,1).isin(df2.apply(tuple,1))]
# # df1.merge(df2,indicator = True, how='left').loc[lambda x : x['_merge']!='both']


In [7]:
# set(stud_info["lvl_of_education"])
# sorted(set(stud_info["what?"]))
# stud_info[stud_info["lvl_of_education"] == 'СП']
# stud_info[stud_info["cipher"] == '01.03.02']
# stud_info[stud_info["what?"] == 0]
# stud_info[pd.isnull(stud_info["study_kind"])]
# stud_info[pd.isnull(stud_info["what?"])]
# stud_info[pd.isnull(stud_info["lvl_of_education"])]
# set(stud_info["category"])
# data[pd.isnull(data["what?"])]
# set(stud_info["cipher"])

In [8]:
stud_info[stud_info["stud"] == 182383]

,stud,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
3582,182383,М,СТ,ДОГ,З,11.03.03,Конструирование и технология электронных средств,966.0,ОСН,"СРЕДНЯЯ ШКОЛА N 18, НОВГОРОД"
3583,182383,М,СТ,ДОГ,З,11.03.03,Конструирование и технология электронных средств,9284.0,СП,Политехнический колледж НовГУ


In [9]:
stud_info

,stud,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
0,99136,Ж,ВЫП,БП,Д,31.05.01,Лечебное дело,2629.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
1,99112,Ж,ВЫП,БП,Д,31.05.01,Лечебное дело,8240.0,СР,"МОУ СОШ N 2, ПЕСТОВО"
2,99109,Ж,ВЫП,БП,Д,31.05.01,Лечебное дело,2314.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
3,99125,Ж,ВЫП,БП,Д,31.05.01,Лечебное дело,2345.0,СР,"Гимназия ""ГАРМОНИЯ"""
4,100676,М,ВЫП,БП,Д,31.05.01,Лечебное дело,2650.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...
5461,100678,Ж,СТ,БП,Д,31.05.01,Лечебное дело,2388.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5462,100663,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,2554.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5463,191630,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,6692.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
5464,191625,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,NaN,NaN,NaN


In [10]:
%%capture
stud_info.fillna({"lvl_of_education": "НЕТ", "what?": 0.0}, inplace=True)

data = data.merge(stud_info, how="left", on="stud")
X_validation = X_validation.merge(stud_info, how="left", on="stud")

In [11]:
# set(data["lvl_of_education"])
data[data["lvl_of_education"].isnull()]

,stud,profession,semester,exam_type,discipline,mark,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education


In [12]:
data

,stud,profession,semester,exam_type,discipline,mark,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education
0,175711,38.03.02 Менеджмент,4.0,Экзамен,Маркетинг,5,М,СТ,БП,Д,38.03.02,Менеджмент,5445.0,СР,"СРЕДНЯЯ ШКОЛА N 31, НОВГОРОД"
1,100647,31.05.01 Лечебное дело,5.0,Экзамен,"Пропедевтика внутренних болезней, лучевая диаг...",5,Ж,ВЫП,ДОГ,Д,31.05.01,Лечебное дело,2668.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
2,199311,31.05.03 Стоматология,1.0,Экзамен,Химия,5,Ж,СТ,БП,Д,31.05.03,Стоматология,11431.0,СР,"ГИМНАЗИЯ N 2, НОВГОРОД"
3,162002,38.03.02 Менеджмент,3.0,Экзамен,Статистика,4,М,СТ,БП,Д,38.03.02,Менеджмент,4981.0,СР,"Гимназия ""ГАРМОНИЯ"""
4,108384,31.05.01 Лечебное дело,9.0,Экзамен,Педиатрия,3,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,2034.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111446,176141,40.03.01 Юриспруденция,2.0,Курсовая работа,Конституционное право,3,Ж,СТ,ДОГ,В,40.03.01,Юриспруденция,8613.0,СР,"МОУ СОШ N 1, СОЛЬЦЫ с филиалом в Куклино"
111447,182794,31.05.01 Лечебное дело,4.0,Экзамен,Нормальная физиология,3,Ж,СТ,ДОГ,Д,31.05.01,Лечебное дело,5505.0,СР,ИНОГОРОДНЯЯ СРЕДНЯЯ ШКОЛА
111448,162391,31.05.01 Лечебное дело,2.0,Дифференцированный зачет,Иностранный язык,4,Ж,СТ,БП,Д,31.05.01,Лечебное дело,4900.0,СР,"ГИМНАЗИЯ N 1, НОВГОРОД"
111449,202479,38.03.01 Экономика,1.0,Дифференцированный зачет,Информатика,4,Ж,СТ,ДОГ,З,38.03.01,Экономика,5466.0,СР,"Гимназия ""ГАРМОНИЯ"""


In [13]:
# set(data["exam_type"])
# set(data["what?"])

In [14]:
# data[data["exam_type"] == "Экзамен"].sort_values(by=["mark"]).groupby(["stud"]).count()

In [15]:

# encode labels
from sklearn import preprocessing

fields = ["discipline", "profession", "exam_type", "sex", "category", "speciality", "education", "state", "cipher"]
le_s = {
    field_name: preprocessing.LabelEncoder().fit(pd.concat([data[field_name], X_validation[field_name]]))
    for field_name in fields}

order = [
    "НЕТ",  # 190  Нет данных
    "ОО",   # 160  Начальное общее образование
    "ОСН",  # 32   Основное общее образование
    "НС",   # 14   Неполное среднее образование
    "СР",   # 4101 Среднее общее образование
    "НВ",   # 2    Неполное высшее образование
    "НП",   # 50   Начальное/Незаконченное? профессиональное образование
    "СП",   # 916  Среднее профессиональное образование
]
le_s["lvl_of_education"] = preprocessing.LabelEncoder().fit(order)

order = ["В", "Д", "З"]  # вечернее, дневное, заочное
le_s["study_kind"] = preprocessing.LabelEncoder().fit(order)

for field_name, le in le_s.items():
    print(field_name)#, set(data[field_name]))
    data[field_name] = le.transform(data[field_name])
    X_validation[field_name] = le.transform(X_validation[field_name])

# 69.0 to 69
fields = ["semester", "what?"]
for field_name in fields:
    data[field_name] = data[field_name].astype(int)
    X_validation[field_name] = X_validation[field_name].astype(int)

# normalize
data["semester"] -= 1
X_validation["semester"] -= 1

discipline
profession
exam_type
sex
category
speciality
education
state
cipher
lvl_of_education
study_kind


In [16]:
# means
fields = ["stud", "profession", "discipline", "speciality", "education", "cipher"]
for field_name in fields:
    mean_mark = data.groupby(field_name).mean()["mark"]

    mean_name = field_name + "_mean"
    data[mean_name] = data[field_name].map(mean_mark)
    X_validation[mean_name] = X_validation[field_name].map(mean_mark)

In [17]:
data

,stud,profession,semester,exam_type,discipline,mark,sex,state,category,study_kind,...,speciality,what?,lvl_of_education,education,stud_mean,profession_mean,discipline_mean,speciality_mean,education_mean,cipher_mean
0,175711,36,3,4,408,5,1,1,0,1,...,19,5445,7,195,4.181818,4.158263,4.352201,4.169750,4.179394,4.158263
1,100647,24,4,4,946,5,0,0,1,1,...,17,2668,7,39,4.116279,4.209998,4.281690,4.209998,4.124297,4.209998
2,199311,25,0,4,1326,5,0,1,0,1,...,29,11431,7,23,4.750000,4.115735,3.565497,4.115735,4.332842,4.115735
3,162002,36,2,4,1100,4,1,1,0,1,...,19,4981,7,32,4.172414,4.158263,3.699422,4.169750,4.289567,4.158263
4,108384,24,8,4,713,3,0,1,1,1,...,17,2034,7,39,3.542857,4.209998,4.336207,4.209998,4.124297,4.209998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111446,176141,40,1,2,364,3,0,1,1,0,...,43,8613,7,71,3.615385,3.948173,3.831884,3.989084,4.068519,3.948173
111447,182794,24,3,4,556,3,0,1,1,1,...,17,5505,7,39,4.052632,4.209998,3.698837,4.209998,4.124297,4.209998
111448,162391,24,1,0,228,4,0,1,0,1,...,17,4900,7,22,4.458333,4.209998,4.144448,4.209998,4.285253,4.209998
111449,202479,35,0,0,238,4,0,1,1,2,...,40,5466,7,32,4.187500,4.107314,3.976378,4.107314,4.289567,4.107314


In [18]:
X_validation

,stud,profession,semester,exam_type,discipline,sex,state,category,study_kind,cipher,speciality,what?,lvl_of_education,education,stud_mean,profession_mean,discipline_mean,speciality_mean,education_mean,cipher_mean
0,163261,29,4,0,260,0,1,0,2,29,1,4972,6,30,4.095238,4.322581,4.190570,4.322581,4.107554,4.322581
1,163261,29,4,0,260,0,1,0,2,29,1,8194,4,192,4.095238,4.322581,4.190570,4.322581,4.177481,4.322581
2,162990,22,6,4,1028,0,1,0,1,22,41,5027,7,39,3.896552,3.729659,3.640000,3.729659,4.124297,3.729659
3,192595,40,1,4,269,1,1,1,1,40,43,6998,7,39,3.909091,3.948173,4.047414,3.989084,4.124297,3.948173
4,198675,35,2,0,230,1,1,1,1,35,40,5525,7,195,4.227273,4.107314,4.104921,4.107314,4.179394,4.107314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27880,170508,40,6,0,41,0,1,1,2,40,43,5182,6,30,3.965517,3.948173,3.617284,3.989084,4.107554,3.948173
27881,178316,25,1,4,141,0,1,1,1,25,29,6285,7,39,3.714286,4.115735,3.930403,4.115735,4.124297,4.115735
27882,186542,51,2,0,228,0,1,0,1,51,12,6423,6,30,3.916667,4.467933,4.144448,4.467933,4.107554,4.467933
27883,186542,51,2,0,228,0,1,0,1,51,12,9087,4,189,3.916667,4.467933,4.144448,4.467933,4.393701,4.467933


In [19]:
data.columns, X_validation.columns

(Index(['stud', 'profession', 'semester', 'exam_type', 'discipline', 'mark',
        'sex', 'state', 'category', 'study_kind', 'cipher', 'speciality',
        'what?', 'lvl_of_education', 'education', 'stud_mean',
        'profession_mean', 'discipline_mean', 'speciality_mean',
        'education_mean', 'cipher_mean'],
       dtype='object'),
 Index(['stud', 'profession', 'semester', 'exam_type', 'discipline', 'sex',
        'state', 'category', 'study_kind', 'cipher', 'speciality', 'what?',
        'lvl_of_education', 'education', 'stud_mean', 'profession_mean',
        'discipline_mean', 'speciality_mean', 'education_mean', 'cipher_mean'],
       dtype='object'))

In [20]:
# data[(data["stud"] == 171886) ].sort_values(by=["semester"])
# data[(data["stud"] == 100647) ].sort_values(by=["semester"])
# data[(data["year"] != data["course"]) ]
# data[data["semester"] == 0 ]

In [21]:
# create dummy variables
columns = []#"exam_type"]#, "discipline", "profession"]
data = pd.get_dummies(data, columns=columns)
X_validation = pd.get_dummies(X_validation, columns=columns)

# remove unneeded data
# use previous fields
# fields = ["stud", "discipline", "profession"]
data.drop(fields, axis=1, inplace=True)
X_validation.drop(fields, axis=1, inplace=True)

In [22]:
data

,semester,exam_type,mark,sex,state,category,study_kind,what?,lvl_of_education,stud_mean,profession_mean,discipline_mean,speciality_mean,education_mean,cipher_mean
0,3,4,5,1,1,0,1,5445,7,4.181818,4.158263,4.352201,4.169750,4.179394,4.158263
1,4,4,5,0,0,1,1,2668,7,4.116279,4.209998,4.281690,4.209998,4.124297,4.209998
2,0,4,5,0,1,0,1,11431,7,4.750000,4.115735,3.565497,4.115735,4.332842,4.115735
3,2,4,4,1,1,0,1,4981,7,4.172414,4.158263,3.699422,4.169750,4.289567,4.158263
4,8,4,3,0,1,1,1,2034,7,3.542857,4.209998,4.336207,4.209998,4.124297,4.209998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111446,1,2,3,0,1,1,0,8613,7,3.615385,3.948173,3.831884,3.989084,4.068519,3.948173
111447,3,4,3,0,1,1,1,5505,7,4.052632,4.209998,3.698837,4.209998,4.124297,4.209998
111448,1,0,4,0,1,0,1,4900,7,4.458333,4.209998,4.144448,4.209998,4.285253,4.209998
111449,0,0,4,0,1,1,2,5466,7,4.187500,4.107314,3.976378,4.107314,4.289567,4.107314


In [23]:
from sklearn.model_selection import train_test_split

marks = data.pop("mark")
X_train, X_test, y_train, y_test = train_test_split(data, marks, shuffle=True, test_size=0.2)

# создание и обучение модели на основе бустинга рещающих деревьев
lgb = lightgbm.LGBMRegressor(n_estimators=1000, verbose=1)

lgb.fit(X_train, y_train)


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.229291 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1180
[LightGBM] [Info] Number of data points in the train set: 89160, number of used features: 14
[LightGBM] [Info] Start training from score 4.175157


LGBMRegressor(n_estimators=1000, verbose=1)

In [24]:
from sklearn.metrics import mean_absolute_error, r2_score

pred_mark = lgb.predict(X_test)
print(mean_absolute_error(y_test, pred_mark))
print(r2_score(y_test, pred_mark))
print(len(pred_mark), len(pd.read_csv("X_test.csv", index_col=0)))

y_pred = pd.read_csv("sample_submission.csv", index_col=0)
pred_mark = lgb.predict(X_validation)
print(len(X_validation), len(pred_mark), y_pred.index)
y_pred["mark"] = pred_mark

pd.DataFrame

y_pred.to_csv("baseline_submission.csv")


0.3891976057441965
0.563856001609885
22291 26265
27885 27885 Int64Index([ 56065, 304637, 177437, 222396, 271996, 296113, 216793, 216914,
             19077, 273892,
            ...
              1083, 211901,  65338, 276526, 200769,  49838, 273753,  35426,
            160476,  18547],
           dtype='int64', name='index', length=26265)


ValueError: Length of values (27885) does not match length of index (26265)